# SemEval 2021 Task 11 sub-task A

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Pre-processing Train and Valid


In [ ]:
!git clone https://github.com/ncg-task/training-data.git "/content/train"

In [ ]:
#### Loading Training Data and store it
input_dir = "/content/train/"
import os
import torch
import numpy as np
import time
from torch.utils.data import Dataset, DataLoader
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
list_of_folders = ["query_wellformedness", "passage_re-ranking", "part-of-speech_tagging", 
         "sentence_compression", "sentiment_analysis", "temporal_information_extraction", 
         "phrase_grounding", "text_generation", "text-to-speech_synthesis", 
         "smile_recognition", "topic_models", "question_generation", 
         "relation_extraction", "paraphrase_generation", "question_similarity", 
         "question_answering", "sentence_classification", "prosody_prediction", 
         "semantic_role_labeling", "text_summarization", "semantic_parsing",
         "sarcasm_detection", "natural_language_inference", "negation_scope_resolution"]
input_stanza_list = []
input_stanza_len = []
input_sent_num_list = []
file_name_list = []
for fls in list_of_folders:
  count=0
  for i in os.listdir(input_dir + fls + '/'):
    count=count+1
    for files in os.listdir(input_dir + fls + '/' + str(i)):
      if files.endswith("Stanza-out.txt"):
        stanza_file = open(input_dir + fls + '/' + str(i) + '/' + files, "r")
        stanza_lines = stanza_file.read()
        stanza_lines_list = list(filter(None,map(lambda x:x.lower(),stanza_lines.splitlines()))) # filter empty strings and split into lines
        input_stanza_len.append(len(stanza_lines_list))
        input_stanza_list.append(stanza_lines_list)
      if files.endswith("sentences.txt"):
        sentence_file = open(input_dir + fls + '/' + str(i) + '/' + files, "r")
        sentence_num_list = list(filter(None,sentence_file.read().splitlines())) # filter empty strings and split into lines
        input_sent_num_list.append(sentence_num_list)
    file_name_list.append(fls + '/' + str(i))
  print("Training complete",fls, count)

In [ ]:
!git clone https://github.com/ncg-task/trial-data.git "/content/valid"

In [ ]:
##### Loading valid dataset
val_input_dir = "/content/valid/"
val_list_of_folders = ["machine-translation", "named-entity-recognition", "question-answering",
         "relation-classification", "text-classification"]
val_input_stanza_list = []
val_input_sent_num_list = []
val_file_name_list = []
val_input_stanza_len = []
for fls in val_list_of_folders:
  count=0
  for i in os.listdir(val_input_dir + fls + '/'):
    count=count+1
    for files in os.listdir(val_input_dir + fls + '/' + str(i)):
      if files.endswith("Stanza-out.txt"):
        stanza_file = open(val_input_dir + fls + '/' + str(i) + '/' + files, "r")
        stanza_lines = stanza_file.read()
        stanza_lines_list = list(filter(None, map(lambda x:x.lower(),stanza_lines.splitlines()))) # filter empty strings and split into lines
        val_input_stanza_len.append(len(stanza_lines_list))
        val_input_stanza_list.append(stanza_lines_list)
      if files.endswith("sentences.txt"):
        sentence_file = open(val_input_dir + fls + '/' + str(i) + '/' + files, "r")
        sentence_num_list = list(filter(None,sentence_file.read().splitlines())) # filter empty strings and split into lines
        val_input_sent_num_list.append(sentence_num_list)
    val_file_name_list.append(fls + '/' + str(i))
  print("Valid complete",fls,count)

In [ ]:
#### string to int 
#### conversion for Train set
input_sent_num_list = [[int(s) for s in sublist] for sublist in input_sent_num_list] # convert sentence list string to integer
input_sent_num_list = [list(set(x)) for x in input_sent_num_list]

#### conversion for Valid set
val_input_sent_num_list = [[int(s) for s in sublist] for sublist in val_input_sent_num_list] # convert sentence list string to integer
val_input_sent_num_list = [list(set(x)) for x in val_input_sent_num_list]

In [ ]:
#### Ground truth label formation for classification
#### Train set
multihot_input_sent = []
for i in range(len(input_stanza_list)):
  temp =[0]*input_stanza_len[i]
  for j in range(len(input_sent_num_list[i])):
    t1 = input_sent_num_list[i][j] -1
    temp[t1] = 1
  multihot_input_sent.append(temp)
#### valid set
val_multihot_input_sent = []
for i in range(len(val_input_stanza_list)):
  temp =[0]*val_input_stanza_len[i]
  for j in range(len(val_input_sent_num_list[i])):
    t1 = val_input_sent_num_list[i][j] -1
    temp[t1] = 1
  val_multihot_input_sent.append(temp)

In [ ]:
## Function for flattening 2d list to 1d list
from collections import Iterable
def flatten(lis):
  for item in lis:
    if isinstance(item, Iterable) and not isinstance(item, str):
      for x in flatten(item):
        yield x
    else:        
      yield item

In [ ]:
#### Flatten the list and use heuristic to remove vague sentences
#### Train set
from collections import Counter

train_sentences = list(flatten(input_stanza_list))
train_label = list(flatten(multihot_input_sent))

train_tuple = list(set((zip(train_sentences,train_label))))
train_in_sentences = []
train_sent_label = []
for stan,lab in train_tuple:
  if len(stan) >4:
    train_in_sentences.append(stan)
    train_sent_label.append(lab)

valid_sentences = list(flatten(val_input_stanza_list))
valid_label = list(flatten(val_multihot_input_sent))

valid_tuple = list(set((zip(valid_sentences,valid_label))))
valid_in_sentences = []
valid_sent_label = []
for stan,lab in valid_tuple:
  if len(stan) >4:
    valid_in_sentences.append(stan)
    valid_sent_label.append(lab)

print("train size",len(train_in_sentences),len(train_sent_label))
print("valid size",len(valid_in_sentences),len(valid_sent_label))

In [ ]:
#### Five timesOversampling of minority class in train set
train_tuple = list(set((zip(train_in_sentences,train_sent_label))))
train_sum_sent = 5 * [stan for stan,label in train_tuple if label==1]
train_nonsum_sent = [stan for stan,label in train_tuple if label==0]
train_in_sentences = train_sum_sent + train_nonsum_sent
train_sent_label = len(train_sum_sent)*[1] + len(train_nonsum_sent)*[0]
print("Oversampled train size",len(train_in_sentences),len(train_sent_label))

# Model Building

In [ ]:
!pip install -q transformers

In [ ]:
########### Library import and random seed
import numpy as np
from sklearn import metrics
import transformers
import torch
import random
from transformers import AutoTokenizer, AutoModel
seed_val = 66
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
#### Hyper-parameters for the model
EPOCHS = 10
LEARNING_RATE = 1e-05
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')			
MAX_LEN = 100
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32

In [ ]:
#### Create Dataset Loader
class Triage(Dataset):
    def __init__(self, data_in,data_out, tokenizer, max_len):
        self.len = len(data_in)
        self.data = data_in
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.label = data_out
        
    def __getitem__(self, index):
        # title = str(self.data.TITLE[index])
        # title = " ".join(title.split())
        title = self.data[index]
        inputs = self.tokenizer.encode_plus(title,
            None,add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True,
            return_length = True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs['token_type_ids']
        lengths = inputs['length']
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'lengths': torch.tensor(lengths, dtype=torch.long),
            'targets': torch.tensor(self.label[index], dtype=torch.long)
        } 
    
    def __len__(self):
      return self.len

In [ ]:
#### Train and Valid Loader 
training_set = Triage(train_in_sentences,train_sent_label, tokenizer, MAX_LEN)
train_params = {'batch_size': TRAIN_BATCH_SIZE,'shuffle': True,'num_workers': 0}
training_loader = DataLoader(training_set, **train_params)
valid_set = Triage(valid_in_sentences,valid_sent_label, tokenizer, MAX_LEN)
valid_params = {'batch_size': VALID_BATCH_SIZE,'shuffle': True,'num_workers': 0}
valid_loader = DataLoader(valid_set, **valid_params)

In [ ]:
# Create a SciBERT + Bi-LSTM model for binary classification
class SCIBERTClass(torch.nn.Module):
    def __init__(self):
        super(SCIBERTClass, self).__init__()
        self.l1 = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased',  output_hidden_states=True)
        self.lstm = torch.nn.LSTM(768, 400, num_layers=2, batch_first = True, bidirectional=True)
        self.l2 = torch.nn.Dropout(0.1)
        self.l3 = torch.nn.Linear( 800, 400)
        self.l4 = torch.nn.Linear(400,100) # check with layer with 30
        self.l5 = torch.nn.Linear(100,2)
    
    def forward(self, ids, mask, token_type_ids, lengths):
        encoded_layers = self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)[2]
        scibert_hidden_layer = encoded_layers[12]
        enc_hiddens, (last_hidden, last_cell) = self.lstm(torch.nn.utils.rnn.pack_padded_sequence(scibert_hidden_layer, lengths, batch_first=True, enforce_sorted=False)) #enforce_sorted=False  #pack_padded_sequence(data and batch_sizes
        output_hidden = torch.cat((last_hidden[0], last_hidden[1]), dim=1)  # (batch_size, 2*hidden_size)
        output_hidden = self.l2(output_hidden)            # no size change
        output_2 = self.l3(output_hidden)
        output_2 = torch.nn.ReLU()(output_2)
        output_4 = self.l4(output_2)
        output_5 = self.l5(output_4)
        return output_5

model = SCIBERTClass()
model.to(device)

In [ ]:
#### Optimizer, Loss function and evaluation metric
optimizer = torch.optim.AdamW(params = model.parameters(), lr=LEARNING_RATE)
c_weights = torch.tensor([list(Counter(train_sent_label).values())[0], list(Counter(train_sent_label).values())[1]], dtype=torch.float32).to(device)
c_weights = 1.0/(c_weights/c_weights.sum())
c_weights = c_weights/c_weights.sum()
loss_function = torch.nn.CrossEntropyLoss(weight = c_weights)
print("class weights",c_weights)
def calcuate_accu(big_idx, targets):
  n_correct = torch.sum((big_idx==targets) * (targets==1)).item()
  return n_correct
def calculate_labels(targets):
  return torch.sum(targets==1).item()

In [ ]:
#### Train one epoch
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    n_predict = 0
    n_ground = 0
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        lengths = data['lengths'].squeeze(1)
        targets = data['targets'].to(device, dtype = torch.long)
        outputs = model(ids, mask,token_type_ids, lengths)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accu(big_idx, targets)
        n_predict +=calculate_labels(big_idx)
        n_ground +=calculate_labels(targets)
        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        if _%1000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 1000 steps: {loss_step}")
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Accuracy Epoch {epoch}: {epoch_accu} and Loss: {epoch_loss}")
    P = n_correct/n_predict if n_predict else 0.0
    R = n_correct/n_ground if n_ground else 0.0
    F1 = (2*P*R)/(P+R) if P+R else 0.0
    print(f"Training P and R and F1 Epoch {epoch}: {P},{R}, {F1}")
    return

In [ ]:
#### Valid one epoch
def valid(epoch,model, valid_loader):
    model.eval()
    tr_loss = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    n_correct = 0
    n_predict = 0
    n_ground = 0
    with torch.no_grad():
        for _, data in enumerate(valid_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            lengths = data['lengths'].squeeze(1)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask,token_type_ids, lengths)
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accu(big_idx, targets)
            n_predict +=calculate_labels(big_idx)
            n_ground +=calculate_labels(targets)
            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Accuracy Epoch {epoch}: {epoch_accu} and Loss: {epoch_loss}")
    P = n_correct/n_predict if n_predict else 0.0
    R = n_correct/n_ground if n_ground else 0.0
    F1 = (2*P*R)/(P+R) if P+R else 0.0
    print(f"Valiadation  P and R and F1 Epoch {epoch}: {P},{R}, {F1}")
    return epoch_loss

In [ ]:
# Run model.
for epoch in range(EPOCHS):
    train(epoch)
    epoch_loss = valid(epoch,model, valid_loader)
    torch.save(model,f"/content/drive/MyDrive/task-1-semeval/scibert-class-A-{epoch}.pt")

# Test Inference

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Ensure same settings which is used at the time of building model
# import libraries
!pip install -q transformers
import transformers
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
import os
import torch
import numpy as np
import time
import random
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
seed_val = 66
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
# Create a SciBERT + Bi-LSTM model for binary classification
class SCIBERTClass(torch.nn.Module):
    def __init__(self):
        super(SCIBERTClass, self).__init__()
        self.l1 = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased',  output_hidden_states=True)
        self.lstm = torch.nn.LSTM(768, 400, num_layers=2, batch_first = True, bidirectional=True)
        self.l2 = torch.nn.Dropout(0.1)
        self.l3 = torch.nn.Linear( 800, 400)
        self.l4 = torch.nn.Linear(400,100) # check with layer with 30
        self.l5 = torch.nn.Linear(100,2)
    
    def forward(self, ids, mask, token_type_ids, lengths):
        encoded_layers = self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)[2]
        scibert_hidden_layer = encoded_layers[12]
        enc_hiddens, (last_hidden, last_cell) = self.lstm(torch.nn.utils.rnn.pack_padded_sequence(scibert_hidden_layer, lengths, batch_first=True, enforce_sorted=False)) #enforce_sorted=False  #pack_padded_sequence(data and batch_sizes
        output_hidden = torch.cat((last_hidden[0], last_hidden[1]), dim=1)  # (batch_size, 2*hidden_size)
        output_hidden = self.l2(output_hidden)            # no size change
        output_2 = self.l3(output_hidden)
        output_2 = torch.nn.ReLU()(output_2)
        output_4 = self.l4(output_2)
        output_5 = self.l5(output_4)
        return output_5

model = SCIBERTClass()
model.to(device)

model = torch.load("/content/drive/MyDrive/task-1-semeval/scibert-class-A-1.pt")
model.eval()

In [ ]:
#%rm -r "/content/drive/MyDrive/submission8/"
%mkdir "/content/drive/MyDrive/output-subtask-A/"
output_dir = "/content/drive/MyDrive/output-subtask-A/"

In [ ]:
%mkdir "/content/drive/MyDrive/submission8/constituency_parsing"
%mkdir "/content/drive/MyDrive/submission8/coreference_resolution"
%mkdir "/content/drive/MyDrive/submission8/data-to-text_generation"
%mkdir "/content/drive/MyDrive/submission8/dependency_parsing"
%mkdir "/content/drive/MyDrive/submission8/document_classification"
%mkdir "/content/drive/MyDrive/submission8/entity_linking"
%mkdir "/content/drive/MyDrive/submission8/face_alignment"
%mkdir "/content/drive/MyDrive/submission8/face_detection"
%mkdir "/content/drive/MyDrive/submission8/hypernym_discovery"
%mkdir "/content/drive/MyDrive/submission8/natural_language_inference"

In [ ]:
# test data
!git clone https://github.com/ncg-task/evaluation-phase1.git "/content/test"

In [ ]:
#####Testing dataset reading
from shutil import copyfile
test_input_dir = "/content/test/"
test_list_of_folders = ["constituency_parsing","coreference_resolution",
                   "data-to-text_generation","dependency_parsing",
                   "document_classification","entity_linking",
                   "face_alignment","face_detection", "hypernym_discovery",
                   "natural_language_inference"]
test_input_stanza_list = []
test_file_name_list = []
test_input_stanza_len = []
MAX_LEN=100
for fls in test_list_of_folders:
  count=0
  for i in os.listdir(test_input_dir + fls + '/'):
    count=count+1
    output_info_folder = os.path.join(output_dir,fls,str(i))
    os.makedirs(output_info_folder)
    for files in os.listdir(test_input_dir + fls + '/' + str(i)):
      if files.endswith("Grobid-out.txt"):
        copyfile(test_input_dir + fls + '/' + str(i) + '/' + files, output_dir + fls + '/' + str(i) + '/' + files)
      if files.endswith("Stanza-out.txt"):
        stanza_file = open(test_input_dir + fls + '/' + str(i) + '/' + files, "r")
        copyfile(test_input_dir + fls + '/' + str(i) + '/' + files, output_dir + fls + '/' + str(i) + '/' + files)
        stanza_lines = stanza_file.read()
        stanza_lines_list = list(filter(None, map(lambda x:x.lower(),stanza_lines.splitlines()))) # filter empty strings and split into lines
        test_input_stanza_len.append(len(stanza_lines_list))
        test_input_stanza_list.append(stanza_lines_list)
    print("size of file",len(stanza_lines_list),fls+'/'+str(i))
    with torch.no_grad():
      output_file=open(os.path.join(output_info_folder, "sentences.txt"),"a")
      if stanza_lines_list[1] != 'abstract':
        output_file.write(str(2) +"\n")
      for t8 in range(2,len(stanza_lines_list)):
        if (stanza_lines_list[t8]=='conclusion') or (stanza_lines_list[t8]=='conclusions'):
          break
        test_input = tokenizer.encode_plus(stanza_lines_list[t8], None, add_special_tokens=True, 
                                           max_length= MAX_LEN, pad_to_max_length=True, 
                                           return_token_type_ids=True, truncation=True, 
                                           return_length = True, return_tensors="pt")
        test_ids = test_input['input_ids'].to(device, dtype = torch.long)
        test_mask = test_input['attention_mask'].to(device, dtype = torch.long)
        test_token_type_ids = test_input['token_type_ids'].to(device, dtype = torch.long)
        lengths = test_input['length']
        outputs = model(test_ids, test_mask, test_token_type_ids, lengths)
        _, output_idx = torch.max(outputs, dim =1)
        t_output_class = output_idx.cpu().detach().numpy().tolist()
        output_file=open(os.path.join(output_info_folder, "sentences.txt"),"a")
        #writing the sentences.txt file showing contribution sentences
        if t_output_class[0] == 1:
          output_file.write(str(t8+1) +"\n")
    test_file_name_list.append(fls + '/' + str(i))
  print("Test complete",fls,count)